In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
!pip install teeplot
from teeplot import teeplot as tp
from matplotlib.patches import Patch






# OSF identifier
data_id = "he5zm"

# load CSV file into DataFrame directly from OSF URL
df = pd.read_csv(f"https://osf.io/{data_id}/download")


In [ ]:


def plot_mean_real_execution_time_ps(data, **kwargs):
    # rename the algorithm labels
    data["algorithm"] = data["Algorithm"].replace({
        "steady_assign_storage_site_batched": "steady",
        "stretched_assign_storage_site_batched": "stretched",
        "tilted_assign_storage_site_batched": "tilted",
    })

    # filter to only include rows where T Upper Bound == 2^32
    data = data[data["T Upper Bound"] == 4294967296]

    # assign S and Tsize directly
    data["S"] = data["Surface Size (S)"]
    data["Tsize"] = data["T Size"]

    # convert to nanoseconds per site
    data["Real Time Per Site (ns)"] = (
        data["Real Execution Time"] * 1e9
        / data["Tsize"]
    )


    # group by algorithm, S and compute mean per-site time
    grouped = data.groupby(["algorithm", "S"], as_index=False)["Real Time Per Site (ns)"].mean()

    # bar plot
    plt.figure(figsize=(3, 2))  # these are IRL dimensions
    ax = sns.barplot(data=grouped, **kwargs, alpha=0.85)

    # Remove top and right spines for cleaner frame
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    # add labels to the bars
    for container in ax.containers:
        labels = [int(v.get_height()) if v.get_height() > 0 else '' for v in container]
        ax.bar_label(container, labels=labels, fmt='%d', fontsize=7, padding=3, rotation=90)

    # better readability
    plt.xlabel("Algorithm", fontsize=8)
    plt.ylabel("Per-Site Real Time (ns)", fontsize=8)

    # keep x-axis labels horizontal
    plt.xticks(fontsize=7, rotation=0)
    plt.yticks(fontsize=8)

    # ylim adjustment
    ax.set_ylim(0, ax.get_ylim()[1] * 1.2)

    # legend
    handles, labels = ax.get_legend_handles_labels()

    # Rebuild square patch handles from the bar containers
    colors = [bar.patches[0].get_facecolor() for bar in ax.containers[:3]]
    color_handles = [Patch(facecolor=c, edgecolor='black') for c in colors]

    # Create a dummy handle for inline title
    title_handle = Patch(facecolor='none', edgecolor='none')
    legend_handles = [title_handle] + color_handles
    legend_labels = [r"$\it{Surface\;Size}$"] + labels  # italic + spacing

    # legend
    plt.legend(
        handles=legend_handles,
        labels=legend_labels,
        fontsize=8,
        loc="lower center",
        bbox_to_anchor=(0.45, 1.02),
        ncol=4,
        columnspacing=1.2,
        handletextpad=0.5,
        frameon=False
    )

    # layout adjustment
    plt.subplots_adjust(bottom=0.4)
    plt.tight_layout(pad=1)


# teeplot
tp.tee(plot_mean_real_execution_time_ps, data=df, x="algorithm", y="Real Time Per Site (ns)", hue="S")
